# Tableau Data Preparation

This notebook transforms the carbon intensity analysis data into formats optimised for Tableau visualisation. The main challenge is converting the wide-format regional data (where each UK region is a separate column) into long format that Tableau can handle effectively for geographic analysis and regional comparisons.

## Key Transformations:
1. **Regional Data Reshaping**: Convert wide format to long format for proper regional analysis
2. **Geographic Categorisation**: Add country and region type classifications
3. **Temporal Features**: Extract hour, day, season for time-based analysis
4. **Export Optimised Datasets**: Create both full and sample datasets for Tableau performance

#### Stage 1: Environment Setup
Import the necessary libraries for data transformation and export.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Libraries imported successfully
Pandas version: 2.3.1
Numpy version: 1.26.4


#### Stage 2: Load the Combined Dataset
Load the processed combined carbon intensity data from our previous analysis.

In [2]:
# Define file paths
project_root = Path().absolute().parent
data_path = project_root / 'data' / 'processed'
output_path = project_root / 'data' / 'processed'

# Load the combined dataset
print("Loading combined carbon intensity dataset...")
combined_df = pd.read_csv(data_path / 'combined_generation_carbon_intensity.csv')

# Convert datetime column
combined_df['datetime'] = pd.to_datetime(combined_df['datetime'])

print(f"Dataset loaded: {combined_df.shape[0]:,} rows × {combined_df.shape[1]} columns")
print(f"Date range: {combined_df['datetime'].min()} to {combined_df['datetime'].max()}")
print(f"Memory usage: {combined_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

Loading combined carbon intensity dataset...
Dataset loaded: 121,034 rows × 48 columns
Date range: 2018-09-17 23:00:00+00:00 to 2025-08-13 11:30:00+00:00
Memory usage: 44.3 MB


#### Stage 3: Regional Data Structure Analysis
Examine the current regional data structure and identify the columns that need reshaping.

In [3]:
# Identify regional columns from the data dictionary
regional_columns = [
    'North Scotland', 'South Scotland', 'North West England', 
    'North East England', 'Yorkshire', 'North Wales and Merseyside',
    'South Wales', 'West Midlands', 'East Midlands', 'East England',
    'South West England', 'South England', 'London', 'South East England'
]

# Verify all regional columns exist in the dataset
missing_regions = [col for col in regional_columns if col not in combined_df.columns]
available_regions = [col for col in regional_columns if col in combined_df.columns]

print(f"Regional columns in dataset: {len(available_regions)}")
print(f"Available regions: {available_regions}")

if missing_regions:
    print(f"Missing regional columns: {missing_regions}")

# Show sample of current wide format
print("\nCurrent wide format structure (first few regional columns):")
sample_cols = ['datetime', 'CARBON_INTENSITY'] + available_regions[:3]
print(combined_df[sample_cols].head())

Regional columns in dataset: 14
Available regions: ['North Scotland', 'South Scotland', 'North West England', 'North East England', 'Yorkshire', 'North Wales and Merseyside', 'South Wales', 'West Midlands', 'East Midlands', 'East England', 'South West England', 'South England', 'London', 'South East England']

Current wide format structure (first few regional columns):
                   datetime  CARBON_INTENSITY  North Scotland  South Scotland  \
0 2018-09-17 23:00:00+00:00              83.0            30.0            11.0   
1 2018-09-17 23:30:00+00:00              83.0            44.0             7.0   
2 2018-09-18 00:00:00+00:00              78.0            44.0             8.0   
3 2018-09-18 00:30:00+00:00              77.0            44.0            11.0   
4 2018-09-18 01:00:00+00:00              76.0            45.0            13.0   

   North West England  
0                38.0  
1                41.0  
2                39.0  
3                37.0  
4                34.0

#### Stage 4: Define Helper Functions
Create functions to categorise regions and prepare data for transformation.

In [4]:
def classify_country(region_name):
    """Classify regions by country (Scotland, Wales, England)"""
    if 'Scotland' in region_name:
        return 'Scotland'
    elif 'Wales' in region_name:
        return 'Wales'
    elif 'England' in region_name or region_name == 'London':
        return 'England'
    else:
        return 'Other'

def classify_region_type(region_name):
    """Classify regions by geographic orientation"""
    if 'North' in region_name:
        return 'Northern'
    elif 'South' in region_name:
        return 'Southern'
    elif 'East' in region_name:
        return 'Eastern'
    elif 'West' in region_name or 'Wales' in region_name:
        return 'Western'
    elif region_name == 'London':
        return 'Metropolitan'
    elif region_name == 'Yorkshire':
        return 'Central'
    else:
        return 'Other'

#### Stage 5: Transform Regional Data to Long Format
Convert the wide format regional data into long format suitable for Tableau analysis.

In [5]:
def create_tableau_regional_dataset(df, regional_cols):
    """Transform wide regional format to long format for Tableau"""
    
    # Identify non-regional columns (these will be ID variables in melt)
    non_regional_cols = [col for col in df.columns if col not in regional_cols]
    
    print(f"Converting {len(regional_cols)} regional columns to long format...")
    print(f"Preserving {len(non_regional_cols)} non-regional columns as identifiers")
    
    # Melt the dataframe to convert wide to long format
    df_long = df.melt(
        id_vars=non_regional_cols,
        value_vars=regional_cols,
        var_name='Region',
        value_name='Regional_Carbon_Intensity'
    )
    
    # Add geographic classifications
    df_long['Country'] = df_long['Region'].apply(classify_country)
    df_long['Region_Type'] = df_long['Region'].apply(classify_region_type)
    
    # Calculate regional deviation from national average
    df_long['Regional_Deviation'] = df_long['Regional_Carbon_Intensity'] - df_long['CARBON_INTENSITY']
    df_long['Regional_Deviation_Percent'] = (df_long['Regional_Deviation'] / df_long['CARBON_INTENSITY']) * 100
    
    return df_long

# Apply the transformation
print("Starting regional data transformation...")
tableau_regional_data = create_tableau_regional_dataset(combined_df, available_regions)

print(f"\nTransformation complete!")
print(f"Original dataset: {combined_df.shape[0]:,} rows × {combined_df.shape[1]} columns")
print(f"Long format dataset: {tableau_regional_data.shape[0]:,} rows × {tableau_regional_data.shape[1]} columns")
print(f"Multiplication factor: {tableau_regional_data.shape[0] // combined_df.shape[0]}x (one row per region per timestamp)")

Starting regional data transformation...
Converting 14 regional columns to long format...
Preserving 34 non-regional columns as identifiers

Transformation complete!
Original dataset: 121,034 rows × 48 columns
Long format dataset: 1,694,476 rows × 40 columns
Multiplication factor: 14x (one row per region per timestamp)


#### Stage 6: Export Datasets for Tableau
Save the transformed datasets in formats optimised for Tableau import.

In [6]:
# Create output directory if it doesn't exist
tableau_output_path = output_path / 'tableau'
tableau_output_path.mkdir(exist_ok=True)

print("Exporting datasets for Tableau...")
print(f"Output directory: {tableau_output_path}")

# Export full regional dataset
full_output_file = tableau_output_path / 'carbon_intensity_regional_long_format.csv'
tableau_regional_data.to_csv(full_output_file, index=False)
file_size_mb = full_output_file.stat().st_size / 1024**2
print(f"Full dataset exported: {full_output_file.name} ({file_size_mb:.1f} MB)")

Exporting datasets for Tableau...
Output directory: /Users/julianelliott/Documents/GitHub/grid-carbon-analysis/data/processed/tableau
Full dataset exported: carbon_intensity_regional_long_format.csv (443.1 MB)


In [7]:
# Fix datetime formatting for Tableau compatibility

# Convert datetime to string format that Tableau can properly parse
tableau_regional_data_fixed = tableau_regional_data.copy()

# Format datetime as ISO string without timezone (Tableau prefers this)
tableau_regional_data_fixed['datetime'] = tableau_regional_data_fixed['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Create separate date and time columns for easier Tableau analysis
tableau_regional_data_fixed['Date_Only'] = pd.to_datetime(tableau_regional_data['datetime']).dt.date
tableau_regional_data_fixed['Time_Only'] = pd.to_datetime(tableau_regional_data['datetime']).dt.strftime('%H:%M:%S')

# Ensure all numeric columns are properly typed
numeric_columns = [
    'CARBON_INTENSITY', 'Regional_Carbon_Intensity', 'Regional_Deviation', 'Regional_Deviation_Percent',
    'GAS', 'COAL', 'NUCLEAR', 'WIND', 'HYDRO', 'IMPORTS', 'BIOMASS', 'SOLAR',
    'GAS_perc', 'COAL_perc', 'NUCLEAR_perc', 'WIND_perc', 'HYDRO_perc', 'RENEWABLE_perc', 'FOSSIL_perc'
]

for col in numeric_columns:
    if col in tableau_regional_data_fixed.columns:
        tableau_regional_data_fixed[col] = pd.to_numeric(tableau_regional_data_fixed[col], errors='coerce')

In [8]:
# Re-parse datetime for temporal feature extraction
temp_datetime = pd.to_datetime(tableau_regional_data_fixed['datetime'])

tableau_regional_data_fixed['Hour'] = temp_datetime.dt.hour
tableau_regional_data_fixed['Day_of_Week'] = temp_datetime.dt.day_name()
tableau_regional_data_fixed['Season'] = temp_datetime.dt.month.map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring', 
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
})
tableau_regional_data_fixed['Time_Period'] = 'Standard'
tableau_regional_data_fixed.loc[tableau_regional_data_fixed['Hour'].between(19, 21), 'Time_Period'] = 'Peak'
tableau_regional_data_fixed.loc[tableau_regional_data_fixed['Hour'].between(2, 4), 'Time_Period'] = 'Off-Peak'

# 1. Create a smaller sample dataset
sample_per_region = 10000  # For initial Tableau development

# Sample evenly across time periods for each region
tableau_sample = []
for region in available_regions:
    region_data = tableau_regional_data_fixed[tableau_regional_data_fixed['Region'] == region]
    if len(region_data) > sample_per_region:
        # Sample evenly across the time period
        sample_indices = np.linspace(0, len(region_data)-1, sample_per_region, dtype=int)
        region_sample = region_data.iloc[sample_indices]
    else:
        region_sample = region_data
    tableau_sample.append(region_sample)

tableau_sample_df = pd.concat(tableau_sample, ignore_index=True)

# 2. Export the sample dataset
sample_file = tableau_output_path / 'carbon_intensity_tableau_sample.csv'
tableau_sample_df.to_csv(sample_file, index=False)
sample_size_mb = sample_file.stat().st_size / 1024**2